In [ ]:
import logging
from datetime import datetime

current_file_name = "5_Replay_Trajectory"

dt_string = datetime.now().strftime("%Y%m%d_%H%M%S")
log_file = f"logs/{current_file_name}/{dt_string}.log"
logging.basicConfig(level=logging.INFO, filename=log_file,filemode="w", format="%(asctime)s %(levelname)s %(message)s")

# https://blog.sentry.io/logging-in-python-a-developers-guide/

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
from helpers.questions import *
from helpers.constants import *
from helpers.pages import *

In [ ]:
pd.set_option('display.max_columns', 500)

In [ ]:
def get_trajecotires_from_csv(csv_df, pages):
    page_names = pages.keys()

    csv_dict = {}

    for page_name in page_names:
        page = csv_df[csv_df["page_name"] == page_name]
        csv_dict[page_name] = page

    return csv_dict

In [ ]:
def exctract_trajectories_from_interactions(path, pages):
    # Traverse through all files in the directory
    folders = os.listdir(path)
    
    interactions_dict = {}

    for folder in folders:
        files = os.listdir(path + "\\" + folder)
        for file in files:
            if file.endswith(".csv"):
                csv_df = pd.read_csv(path + "\\" + folder + "\\" + file)

                # Get number from the folder name
                number = folder.split("_")[1]
                answers_dict = get_trajecotires_from_csv(csv_df, pages)
                interactions_dict[number] = answers_dict
    return interactions_dict
                

In [ ]:
path_fg = 'data\\3_UXtweak_Mouse_Data_Processing\\FG'
path_h = 'data\\3_UXtweak_Mouse_Data_Processing\\H'

extracted_fg = exctract_trajectories_from_interactions(path_fg, pages)
extracted_fg["group"] = "FG"

extracted_h = exctract_trajectories_from_interactions(path_h, pages)
extracted_h["group"] = "H"

In [ ]:
extracted_fg["26"]["page_1"]

In [ ]:
def get_trajectory_from_page(page_df):
    page_df = page_df[["x", "y", "accurate_timestamp", "type"]]
    return page_df

In [ ]:
def get_widt_and_height(page_df):
    return page_df["pageview_screenWidth"].iloc[0], page_df["pageview_screenHeight"].iloc[0]

In [ ]:
def normalize_time(page_df):
    start_time = page_df["accurate_timestamp"].iloc[0]
    page_df["accurate_timestamp"] = page_df["accurate_timestamp"] - start_time
    return page_df

In [ ]:
page = extracted_fg["26"]["page_5"]

In [ ]:
width, height = get_widt_and_height(page)
page_df = get_trajectory_from_page(page)
page_df = normalize_time(page_df)

In [ ]:
# Recalculate height - correct height is screen height - y
page_df["y"] = height - page_df["y"]

In [ ]:
# Draw the trajectory of the mouse on the page using plotly express
# Use screen width and height as max x and y axis
# Use x, y, accurate_timestamp as the data
# Make it an animation over time

clicks_df = page_df[page_df["type"] == "click"]

page_df["Sekundy"] = page_df["accurate_timestamp"] / 1000

# Create the scatter plot
fig = px.scatter(page_df, x='x', y='y', color='Sekundy', range_x=[0, width], range_y=[0, height], color_continuous_scale='Bluered')

# Update layout to set the size and disable axis labels
fig.update_layout(
    xaxis_title=None,  # Disable x-axis label
    yaxis_title=None,  # Disable y-axis label
    xaxis=dict(tickfont=dict(size=16)),  # Set font size of x-axis ticks
    yaxis=dict(tickfont=dict(size=16)),  # Set font size of y-axis ticks
    legend=dict(font=dict(size=16), x=0.05, y=0.95)  # Set font size of legend and position it inside the plot
)

# Customize the color gradient
fig.update_traces(marker=dict(colorscale='Bluered'))

# Add title
fig.update_layout(title='Vizualizácia trajektórie pohybu myšou participanta FG 26 na obrazovke 5', title_x=0.5, title_font=dict(size=24))

# Add big red points from clicks_df
fig.add_trace(go.Scatter(
    x=clicks_df['x'],
    y=clicks_df['y'],
    mode='markers',
    marker=dict(color='green', size=15),
    name='Kliknutie'
))

# Tighten the layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=40))

# Show the plot
fig.show()

In [ ]:
# Make animation
fig = px.scatter(page_df, x='x', y='y', animation_frame='accurate_timestamp', range_x=[0, width], range_y=[0, height])
fig.show()

In [ ]:
# Add difference column to the dataframe between timestamps
page_df["diff"] = page_df["accurate_timestamp"].diff()
# Calculate mean and median diff
print(page_df["diff"].mean())
print(page_df["diff"].median())

In [ ]:
# Encode the type column to be boolean (click = 1, move = 0), name it as click
page_df["click"] = page_df["type"].apply(lambda x: 1 if x == "click" else 0)
page_df = page_df.drop(columns=["type", "diff"])

In [ ]:
page_df

In [ ]:
def normalize_trajectory(page_df, time_interval):
    """
    Normalize the trajectory data to have uniform time intervals and interpolate missing points.

    Args:
    df (pd.DataFrame): DataFrame containing columns 'x', 'y', and 'accurate_timestamp' in milliseconds.
    time_interval (int): Desired time interval in milliseconds between points.

    Returns:
    pd.DataFrame: DataFrame with normalized and interpolated trajectory data.
    """

    df = page_df.copy(deep=True)

    # Set 'accurate_timestamp' as the index
    df.set_index('accurate_timestamp', inplace=True)

    # Convert the index to datetime (this facilitates resampling)
    df.index = pd.to_datetime(df.index, unit='ms')

    # Resample the DataFrame at the specified time interval and interpolate
    df_resampled = df.resample(f'{time_interval}ms').mean()  # 'mean' is a placeholder to allow resampling
    df_interpolated = df_resampled.interpolate(method='linear')

    # Convert the datetime index back to integer timestamps
    df_interpolated.index = df_interpolated.index.astype(np.int64) // 10**6  # Convert back to milliseconds

    return df_interpolated.reset_index()


In [ ]:
page_df_interpolated = normalize_trajectory(page_df, 100)

In [ ]:
page_df_interpolated

In [ ]:
# Convert ms to seconds
page_df_interpolated["seconds"] = page_df_interpolated["accurate_timestamp"] / 1000

# Make animation
fig = px.scatter(page_df_interpolated, x='x', y='y', animation_frame='seconds', range_x=[0, width], range_y=[0, height])
fig.show()